In [4]:
# USAGE
# python social_distance_detector.py --input pedestrians.mp4
# python social_distance_detector.py --input pedestrians.mp4 --output output.avi

# import the necessary packages
from pyimagesearch import social_distancing_config as config
from pyimagesearch.detection import detect_people
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os


In [ ]:
# construct the argument parse and parse the arguments
# --input: The path to the optional video file. If no video file path is provided, your computer’s first webcam will be used by default.
#--output : The optional path to an output (i.e., processed) video file. If this argument is not provided, the processed video will not be exported to disk.
#--display   : By default, we’ll display our social distance application on-screen as we process each frame. Alternatively, you can set this value to 0to process the stream in the background.


ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", type=str, default="",
	help="path to (optional) input video file")
ap.add_argument("-o", "--output", type=str, default="",
	help="path to (optional) output video file")
ap.add_argument("-d", "--display", type=int, default=1,
	help="whether or not output frame should be displayed")
args = vars(ap.parse_args(["--input","/C:/Users/Goyal/pedestrians.mp4","--output","output.avi","--display","1"]))

#--------------------------------------------------------------------------------------------------------------------#

# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([config.MODEL_PATH, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([config.MODEL_PATH, "yolov3.weights"])
configPath = os.path.sep.join([config.MODEL_PATH, "yolov3.cfg"])
#---------------------------------------------------------------------------------------------------------------------#
#Using OpenCV’s DNN module, we load our YOLO net into memory.
#If you have the USE_GPU option set in the config, then the backend processor is set to be your NVIDIA CUDA-capable GPU.
#If you don’t have a CUDA-capable GPU, ensure that the configuration option is set to False so that your CPU is the processor used.


# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
# check if we are going to use GPU
if config.USE_GPU:
	# set CUDA as the preferable backend and target
	print("[INFO] setting preferable backend and target to CUDA...")
	net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
	net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
#----------------------------------------------------------------------------------------------------------------------#
#Here,gather the output layer names from YOLO; we’ll need them in order to process our results.
#We then start our video stream (either a video file via the --input command line argument or a webcam stream) 
#For now, we initialize our output video writer to None .Further setup occurs in the frame processing loop.
#Finally, we’re ready to begin processing frames and determining if people are maintaining safe social distance:

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream and pointer to output video file
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
writer = None
#----------------------------------------------------------------------------------------------------------------------#
#begins a loop over frames from our video stream.
#The dimensions of our input video for testing are quite large, so we resize each frame while maintaining aspect ratio
#Using our detect_people function implemented in the previous section, we grab results of YOLO object detection. If you need a refresher on the input parameters required or the format of the output results for the function call, be sure to refer to the listing in the previous section.
#We then initialize our violate set. this set maintains a listing of people who violate social distance regulations set forth by public health professionals.



# loop over the frames from the video stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break

	# resize the frame and then detect people (and only people) in it
	frame = imutils.resize(frame, width=700)
	results = detect_people(frame, net, ln,
		personIdx=LABELS.index("person"))

	# initialize the set of indexes that violate the minimum social
	# distance
	violate = set()
#-------------------------------------------------------------------------------------------------------------------#


#Assuming that at least two people were detected in the frame  we proceed to:
#Compute the Euclidean distance between all pairs of centroids 
#Loop over the upper triangular of distance matrix (since the matrix is symmetrical) 
#Check to see if the distance violates our minimum social distance set forth by public health professionals .If two people are too close, we add them to the violate set



	# ensure there are *at least* two people detections (required in
	# order to compute our pairwise distance maps)
	if len(results) >= 2:
		# extract all centroids from the results and compute the
		# Euclidean distances between all pairs of the centroids
		centroids = np.array([r[2] for r in results])
		D = dist.cdist(centroids, centroids, metric="euclidean")

		# loop over the upper triangular of the distance matrix
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				# check to see if the distance between any two
				# centroid pairs is less than the configured number
				# of pixels
				if D[i, j] < config.MIN_DISTANCE:
					# update our violation set with the indexes of
					# the centroid pairs
					violate.add(i)
					violate.add(j)
#---------------------------------------------------------------------------------------------------------------------#
# Looping over the results, we proceed to:
#Extract the bounding box and centroid coordinates 
#Initialize the color of the bounding box to green 
#Check to see if the current index exists in our violate set, and if so, update the color to red
# Draw both the bounding box of the person and their object centroid . Each is color -coordinated, so we’ll see which people are too close.
# Display information on the total number of social distancing violations (the length of our violate set (Lines 108-110)

	# loop over the results
	for (i, (prob, bbox, centroid)) in enumerate(results):
		# extract the bounding box and centroid coordinates, then
		# initialize the color of the annotation
		(startX, startY, endX, endY) = bbox
		(cX, cY) = centroid
		color = (0, 255, 0)

		# if the index pair exists within the violation set, then
		# update the color
		if i in violate:
			color = (0, 0, 255)

		# draw (1) a bounding box around the person and (2) the
		# centroid coordinates of the person,
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.circle(frame, (cX, cY), 5, color, 1)

	# draw the total number of social distancing violations on the
	# output frame
	text = "Social Distancing Violations: {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)
#--------------------------------------------------------------------------------------------------------------------#
#To close out, we:
#Display the frame to the screen if required while waiting for the q (quit) key to be pressed 
#Initialize our video writer if necessary 
#Write the processed (annotated) frame to disk 


	# check to see if the output frame should be displayed to our
	# screen
	if args["display"] > 0:
		# show the output frame
		cv2.imshow("Frame", frame)
		key = cv2.waitKey(1) & 0xFF

		# if the `q` key was pressed, break from the loop
		if key == ord("q"):
			break

	# if an output video file path has been supplied and the video
	# writer has not been initialized, do so now
	if args["output"] != "" and writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 25,
			(frame.shape[1], frame.shape[0]), True)

	# if the video writer is not None, write the frame to the output
	# video file
	if writer is not None:
		writer.write(frame)



[INFO] loading YOLO from disk...
[INFO] accessing video stream...
